<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/OpenMined/PipelineDP/blob/main/utility_analysis/examples/Utility_Analysis_for_PipelineDP.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/OpenMined/PipelineDP/blob/main/utility_analysis/examples/Utility_Analysis_for_PipelineDP.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Background

## About this colab

This colab demontrates the Utility Analysis functionality of PipelineDP.

In brief, the analysis aims to answer two questions:
1. **Tuning**: What are good *hyper parameters* for a differentially-private (DP) pipeline?
1. **Analysis**: What is the *error* introduced by DP processing for these hyper parameters?

**Disclaimers:**
* the result of the analysis itself are not differentially-private, which means they need to be handled with care and, normally, they should not be shared broadly.
* this colab was only tested with Google Colab. You can open this colab in Google Colab with by clicking the button in the top.

## Audience
The colab aims to be self-contained. However, the codelab mainly targets users who are familiar with basic knowledge of [Differential Privacy](https://en.wikipedia.org/wiki/Differential_privacy) (DP) and have one of the following goals:

1. Understand how DP affects the result.
1. Tune DP parameters to get the best data utility for a given level of user privacy.

We recommend all readers to take a look at the Background section of the PipelineDP [restaurant visits example]((https://github.com/OpenMined/PipelineDP/blob/main/examples/restaurant_visits.ipynb). It contains the key definitions used across this colab.

## Utility Analysis

The quality of differentially-private output might deteriorate due to several factors and depends on several parameters.

In this Colab we consider two main factors:
1. Adding noise (e.g. using [Laplace mechanism](https://en.wikipedia.org/wiki/Additive_noise_differential_privacy_mechanisms))
1. Limiting maximum contribution of a single individual

The scale of the error introduced by noise is well-known and independent of the data. The effect of contribution bounding is more complicated.

## Why contribution bounding is required

[Laplace and Gaussian mechanisms](https://en.wikipedia.org/wiki/Additive_noise_differential_privacy_mechanisms) require that adding one individual to the input dataset has limited effect on the output. The maximum change is called *sensitivity*.

## Why contribution bounding is complicated

In a real dataset,  one inidividual can often contribute arbitrary large number of data records. In order to limit sensitivity, PipelineDP performs contribution bounding by sampling records per user, which leads to dropping some data. The error introduced by this procedure depends on the input data.

This presents a **bias-variance tradeoff**:

1. Larger contribution bounding parameters lead to more data kept but larger noise error
1. Smaller contribution bounding parameters lead to less data kept, smaller noise error.

Contribution bounding error intoduces bias, while Laplace (or Gaussian) noise introduces variance.

## What PipelineDP analysis does

The analysis has two parts:

1. **Utility Analysis** computes the error introduced by DP processing for a fixed dataset and hyper-parameters;
1. **Parameter tuning** recommends hyper-parameters to make error smaller using utility analysis under the hood.


**Note:** The recommended parameters would not necessarily be optimal. Since the analysis is designed for large distributed datasets and can be expensive, the goal is to find hyper-parameters that are "good enough" but not necessarily optimal.



# Code

This colab uses a fake dataset of restaurant visits. Here's the plan:

1. Install dependencies and download data;
1. Load the data into a Pandas DataFrame; feel free to play with the data;
1. Demonstrate how to apply DP with PipelineDP. Show comparison of the raw and differentially-private counts.
1. Show how to use Utility Analysis to find good hyper parameters.

To perform these steps, run the cells below.


In [ ]:
#@title Install dependencies and download data
# !pip install pipeline-dp==0.2.2rc2
!git clone https://github.com/OpenMined/PipelineDP.git
!pip install python-dp~=1.1.5rc4
import sys
sys.path.append("PipelineDP")

#Download restaurant dataset from github
!wget https://raw.githubusercontent.com/google/differential-privacy/main/examples/go/data/week_data.csv

from IPython.display import clear_output
clear_output()

from dataclasses import dataclass
import pipeline_dp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load and inspect the data

The dataset used in this Colab is a simulated dataset of visits to some restaurant during a 7-day period. The code below loads the dataset in a DataFrame and prints some of its records.

In [ ]:
#@title Read data
df = pd.read_csv('week_data.csv')
df.rename(inplace=True, columns={'VisitorId' : 'user_id', 'Time entered' : 'enter_time', 'Time spent (minutes)' : 'spent_minutes', 'Money spent (euros)' : 'spent_money', 'Day' : 'day'})
data = [index_row[1] for index_row in df.iterrows()]
df.head()

Our goal is to compute the number of visits for each day of the week in a differentially-private manner. In PipelineDP terminology, a "day of week" represents a data *partition*.

Contribution bounding paramers:

- **max_partitions_contributed** : maximum days per week one visitor can eats in a restaurant

- **max_contributions_per_partition**: maximum times per day can a visitor eat in a restaurant


## Computing DP count

In [ ]:
#@title DP parameters and Hyper parameters
@dataclass
class DPBudget:
  epsilon: float
  delta: float = 0

In [ ]:
#@title Hyper parameters
@dataclass
class HyperParameters:
  noise_kind: pipeline_dp.NoiseKind # Laplace or Gaussian
  max_partitions_contributed: int
  max_contributions_per_partition: int

In [ ]:
#@title Function compute_counts_with_dp and visualising (code is long, can be skipped)

def get_data_extractors():
  # Define privacy ID, partition key and aggregated value extractors.
  # The aggregated value extractor isn't used in this example.
  return pipeline_dp.DataExtractors(
    partition_extractor=lambda row: row.day,
    privacy_id_extractor=lambda row: row.user_id,
    value_extractor=lambda row: 1)

def compute_counts_with_dp(rows:list, budget: DPBudget, params: HyperParameters):
  # Set the backend to local backend. Other options (Beam or Spark)
  # are possible.
  backend = pipeline_dp.LocalBackend()

  # Define the total budget.
  budget_accountant = pipeline_dp.NaiveBudgetAccountant(total_epsilon=budget.epsilon, total_delta=budget.delta)

  # Create DPEngine which will execute the logic.
  dp_engine = pipeline_dp.DPEngine(budget_accountant, backend)

  data_extractors = get_data_extractors()
  # Configure the aggregation parameters.
  params = pipeline_dp.AggregateParams(
    noise_kind=params.noise_kind,
    # This example computes only count but we can compute multiple
    # ... metrics at once.
    metrics=[pipeline_dp.Metrics.COUNT],
    # Limits visits contributed by a visitor. A visitor can contribute to
    # ... up to 3 days
    max_partitions_contributed=params.max_partitions_contributed,
    # ... and up to 2 visits per day.
    max_contributions_per_partition=params.max_contributions_per_partition)
  # Configure the output partition keys as they are publicly known.
  # The output should include all week days.
  public_partitions=list(range(1, 8))

  # Create a computational graph for the aggregation.
  # All computations are lazy. dp_result is iterable, but iterating it would
  # fail until budget is computed (below).
  # It’s possible to call DPEngine.aggregate multiple times with different
  # metrics to compute.
  dp_result = dp_engine.aggregate(rows, params, data_extractors, public_partitions)

  # Compute budget per each DP operation.
  budget_accountant.compute_budgets()

  # Here's where the lazy iterator initiates computations and gets transformed
  # into actual results
  return list(dp_result)

def plot_comparison(dp_result):
  non_dp_count = [0] * 7
  days = range(1, 7)
  for row in data:
    index = row['day'] - 1
    non_dp_count[index] += 1

  # Copy the DP result to a list
  dp_count = [0] * 7
  for count_sum_per_day in dp_result:
    index =  count_sum_per_day[0] - 1
    dp_count[index] = count_sum_per_day[1][0]

  days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
  x = np.arange(len(days))

  width = 0.35
  fig, ax = plt.subplots()
  rects1 = ax.bar(x - width/2, non_dp_count, width, label='non-DP')
  rects2 = ax.bar(x + width/2, dp_count, width, label='DP')
  ax.set_ylabel('Visit count')
  ax.set_title('Count visits per day')
  ax.set_xticks(x)
  ax.set_xticklabels(days)
  ax.legend()
  fig.tight_layout()
  plt.show()

In [ ]:
# Let's set some DP budget
dp_budget = DPBudget(epsilon=1.0, delta=1e-8)

In [ ]:
# Let us try with different hyper-parameters (not the output is randomized)
hyper_params = HyperParameters(noise_kind = pipeline_dp.NoiseKind.LAPLACE,
                               max_partitions_contributed=1,
                               max_contributions_per_partition=1)

plot_comparison(compute_counts_with_dp(data, dp_budget, hyper_params))

The DP counts are way smaller than the actual counts. This suggests a lot of data records are dropped because of aggressive contribution bounding.

Let's try larger paramers:

In [ ]:
hyper_params = HyperParameters(noise_kind = pipeline_dp.NoiseKind.LAPLACE,
                               max_partitions_contributed=10,
                               max_contributions_per_partition=10)

plot_comparison(compute_counts_with_dp(data, dp_budget, hyper_params))

The DP counts are closer to actual counts, but maybe we can do better. Let's use the analysis functionality for that!

## PipelineDP Analysis

In [ ]:
#@title run_analysis function (unfold to see implementation details)
import analysis
from analysis import parameter_tuning
from pipeline_dp.dataset_histograms import computing_histograms

@dataclass
class AnalysisResult:
  recommended_hyper_params: HyperParameters
  average_rmse_per_partition: float

def run_analysis(rows, budget: DPBudget):
  # We now that output keys are from 1 to 7 (Mon-Sun). Let us set them as
  # public partitions
  public_partitions=list(range(1, 8))

  # backend and data_extractors specified the same as for DP count.
  backend = pipeline_dp.LocalBackend()
  data_extractors = get_data_extractors()

  # At first compute contirbution_histograms, which contains some statistics
  # which are used by parameter tuning for finding hyper-parameters candidates.
  contribution_histograms = computing_histograms.compute_dataset_histograms(
            rows, data_extractors, backend)
  contribution_histograms = list(contribution_histograms)[0]

  # Specify parameters for tuning

  # For now only optimizing Absolute error is supported.
  minimizing_function = parameter_tuning.MinimizingFunction.ABSOLUTE_ERROR

  # Which contribution bounding parameters to tune.
  parameters_to_tune = parameter_tuning.ParametersToTune(
        max_partitions_contributed=True, max_contributions_per_partition=True)

  aggregate_params = pipeline_dp.AggregateParams(
        noise_kind=pipeline_dp.NoiseKind.LAPLACE, # not used by tuning, required by PipelineDP validation
        metrics=[pipeline_dp.Metrics.COUNT],
        max_partitions_contributed=1, # not used by tuning, required by PipelineDP validation
        max_contributions_per_partition=1 # not used by tuning, required by PipelineDP validation
  )
  tune_options = parameter_tuning.TuneOptions(
        epsilon=budget.epsilon,
        delta=budget.delta,
        aggregate_params=aggregate_params,
        function_to_minimize=minimizing_function,
        parameters_to_tune=parameters_to_tune)

  result, _ = parameter_tuning.tune(rows, backend, contribution_histograms,
                                                  tune_options, data_extractors,
                                                  public_partitions)
  result = list(result)[0] # result is lazy iterator, converting to list enforce run.
  index_best = result.index_best

  # Extract the recommended parameters.
  # Result contains information about all hyper-parameters that were tried.
  noise_kind = result.utility_analysis_parameters.noise_kind[index_best]
  max_partitions_contributed = result.utility_analysis_parameters.max_partitions_contributed[index_best]
  max_contributions_per_partition = result.utility_analysis_parameters.max_contributions_per_partition[index_best]

  # Extract utility information
  utility_report_for_best_parameters = result.utility_reports[index_best]
  rmse_for_best_parameters = utility_report_for_best_parameters.metric_errors[0].absolute_error.rmse
  clear_output()
  return AnalysisResult(HyperParameters(noise_kind, max_partitions_contributed, max_contributions_per_partition), average_rmse_per_partition=rmse_for_best_parameters)

In [ ]:
# Run analysis
recommended_hyper_parameters = run_analysis(data, dp_budget).recommended_hyper_params

print("Recommended parameters:")
print(f"  noise={recommended_hyper_parameters.noise_kind.value}")
print(f"  max_partitions_contributed={recommended_hyper_parameters.max_partitions_contributed}")
print(f"  max_contributions_per_partition={recommended_hyper_parameters.max_contributions_per_partition}")

Recommended parameters:
  noise=laplace
  max_partitions_contributed=4
  max_contributions_per_partition=1


In [ ]:
# Try recommended parameters
plot_comparison(compute_counts_with_dp(data, dp_budget, recommended_hyper_parameters))

DP counts are now much closer to actual counts! If you like to play with different parameters further, you can do so in the next cell.

In [ ]:
#@title Inspect different paramters
noise = "Laplace" # @param ["Laplace", "Gaussian"]
max_partitions_contributed = 1 #@param { type: "number" }
max_contributions_per_partition = 1 #@param { type: "number" }

hyper_params = HyperParameters(noise_kind = pipeline_dp.NoiseKind.LAPLACE,
                               max_partitions_contributed=max_partitions_contributed,
                               max_contributions_per_partition=max_contributions_per_partition)

plot_comparison(compute_counts_with_dp(data, dp_budget, hyper_params))

# Conclusion

We showed how to use PipelineDP to perform differentially-private statistics, understand the data quality, and improve it by choosing better hyper-parameters.

**Go bigger.** PipelineDP shines with large, distributed, many terrabytes-sized datasets. Since the example dataset was very small, we used the local (in-memory) computation. For much larger datasets, you can use PipelineDP to perform calculations on top of Apache Spark or Apache Beam. The complexity is abstracted away, so the main code wouldn't be that much different.

If you like to try PipelineDP analysis on a large distributed dataset, feel free to contact us by dp-open-source@google.com. We're happy to help!